<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/China_A_share_market_tushare.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Quantitative trading in China A stock market with FinRL

Install FinRL

In [1]:
# !pip install wrds
# !pip install swig
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


Install other libraries

In [2]:
# !pip install stockstats
# !pip install tushare
# #install talib
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
# !tar xvzf ta-lib-0.4.0-src.tar.gz
# import os
# os.chdir('ta-lib') 
# !./configure --prefix=/usr
# !make
# !make install
# #!sudo make install # Sometimes it need root 
# os.chdir('../')
# !pip install TA-Lib

In [3]:
# %cd /
# !git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
# %cd /FinRL-Meta/

## Import Modules

In [4]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd 
from IPython import display

# display.set_matplotlib_formats("svg")

from meta import config 
from meta.data_processor import DataProcessor, DataSource
# from main import check_and_make_directories 
from meta.data_processors.tushare import Tushare, ReturnPlotter 
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv 
from agents.stablebaselines3_models import DRLAgent 
import os 
from typing import List 
from argparse import ArgumentParser 
from meta import config 
from meta.config_tickers import DOW_30_TICKER 
from meta.config import ( DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, INDICATORS, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, TRADE_START_DATE, TRADE_END_DATE, ERL_PARAMS, RLlib_PARAMS, SAC_PARAMS, ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL, )

import pyfolio
from pyfolio import timeseries

pd.options.display.max_columns = None

print("ALL Modules have been imported!")

ALL Modules have been imported!


## Create Folders

In [5]:
import os

''' 
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results") 
'''

# check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results")

## Download data, cleaning and feature engineering

In [6]:
ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

TRAIN_START_DATE = '2015-01-01' 
TRAIN_END_DATE= '2019-08-01' 
TRADE_START_DATE = '2019-08-01' 
TRADE_END_DATE = '2020-01-03'

TIME_INTERVAL = "daily" 
kwargs = {} 
# kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5' 
p = DataProcessor(data_source=DataSource.akshare, start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL, **kwargs)

DataSource.akshare successfully connected


### Download and Clean

In [7]:
p.download_data(ticker_list=ticker_list)
p.clean_data()
p.fillna()

100%|██████████| 15/15 [01:13<00:00,  4.92s/it]


Index(['日期', '股票代码', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额',
       '换手率', 'tic'],
      dtype='object')
Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (17960, 8)
Shape of DataFrame:  (18315, 8)


### Add technical indicator

In [8]:
p.add_technical_indicator(config.INDICATORS) 
p.fillna()

#print(f"p.dataframe: {p.dataframe}")
# p.dataframe["date"] = p.dataframe["time"]

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (18270, 17)


In [9]:
import pickle

with open("china_data.pkl", mode="wb") as f:
    pickle.dump(p, f)

## Split training dataset

In [10]:
import pickle

with open("china_data.pkl", mode="rb") as f:
    p = pickle.load(f)

In [11]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE) 

print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 15


In [12]:
print(f"train.tic.unique(): {train.tic.unique()}")

train.tic.unique(): ['600000' '600009' '600016' '600028' '600030' '600031' '600036' '600050'
 '600104' '600196' '600276' '600309' '600519' '600547' '600570']


In [13]:
print(f"train.head(): {train.head()}")

train.head():       tic        time  index   open   high    low  close  adjusted_close  \
0  600000  2015-01-08     45  15.87  15.88  15.20  15.25           15.25   
0  600009  2015-01-08     46  20.18  20.18  19.73  20.00           20.00   
0  600016  2015-01-08     47  10.61  10.66  10.09  10.20           10.20   
0  600028  2015-01-08     48   7.09   7.41   6.83   6.85            6.85   
0  600030  2015-01-08     49  36.40  36.70  34.68  35.25           35.25   

      volume      macd    boll_ub    boll_lb     rsi_30      cci_30  \
0  3306272.0 -0.032571  16.617911  15.012089   6.058641 -125.593009   
0   198117.0 -0.016008  20.663897  19.736103  12.828915  -90.842491   
0  4851684.0 -0.018247  10.957604   9.997396  11.862558  -99.887006   
0  8190902.0 -0.008227   7.342000   6.743000  27.409248   36.578171   
0  6376269.0  0.032910  36.576444  33.808556  61.517448   47.947020   

        dx_30  close_30_sma  close_60_sma  
0   20.602022       15.8150       15.8150  
0  100.000000 

In [14]:
print(f"train.shape: {train.shape}")

train.shape: (16695, 17)


In [15]:
stock_dimension = len(train.tic.unique()) 
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1 

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


## Train

In [16]:
train.columns

Index(['tic', 'time', 'index', 'open', 'high', 'low', 'close',
       'adjusted_close', 'volume', 'macd', 'boll_ub', 'boll_lb', 'rsi_30',
       'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma'],
      dtype='object')

In [17]:
env_kwargs = {
    "stock_dim": stock_dimension,
    "hmax": 1000,
    "initial_amount": 1000000,
    "buy_cost_pct": 6.87e-5,
    "sell_cost_pct": 1.0687e-3,
    "reward_scaling": 1e-4,
    "state_space": state_space,
    "action_space": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "print_verbosity": 1,
    "initial_buy": True,
    "hundred_each_trade": True 
}

train["date"] = train["time"]
e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [18]:
env_train, _ = e_train_gym.get_sb_env() 

print(f"print(type(env_train)): {print(type(env_train))}")

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
print(type(env_train)): None


### DDPG

In [19]:
agent = DRLAgent(env=env_train) 
DDPG_PARAMS = { "batch_size": 256, "buffer_size": 50000, "learning_rate": 0.0005, "action_noise": "normal", } 
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300])) 
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name='ddpg', total_timesteps=10000)

{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device
Logging to tensorboard_log/ddpg\ddpg_3
Episode: 2
day: 1112, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1603481.71
total_reward: 603481.71
total_cost: 12310.29
total_trades: 1346
Sharpe: 0.512
Episode: 3
day: 1112, episode: 3
begin_total_asset: 1000000.00
end_total_asset: 1562122.59
total_reward: 562122.59
total_cost: 130.41
total_trades: 40
Sharpe: 0.509
Episode: 4
day: 1112, episode: 4
begin_total_asset: 1000000.00
end_total_asset: 1663692.64
total_reward: 663692.64
total_cost: 130.36
total_trades: 39
Sharpe: 0.564
Episode: 5
day: 1112, episode: 5
begin_total_asset: 1000000.00
end_total_asset: 1572253.59
total_reward: 572253.59
total_cost: 130.41
total_trades: 37
Sharpe: 0.505
-----------------------------------
| time/              |

### A2C

In [20]:
agent = DRLAgent(env=env_train) 
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c', total_timesteps=50000)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c\a2c_3
----------------------------------------
| time/                 |              |
|    fps                | 387          |
|    iterations         | 100          |
|    time_elapsed       | 1            |
|    total_timesteps    | 500          |
| train/                |              |
|    entropy_loss       | -21.5        |
|    explained_variance | -81.8        |
|    learning_rate      | 0.0007       |
|    n_updates          | 99           |
|    policy_loss        | -3.29        |
|    reward             | -0.017931068 |
|    std                | 1.01         |
|    value_loss         | 0.211        |
----------------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 392         |
|    iterations         | 200         |
|    time_elapsed       | 2           |
|    total_timesteps    | 1000  

## Trade

In [21]:
trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE) 
trade["date"] = trade["time"]

env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": False, "hundred_each_trade": True } 
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

In [22]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)

Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1088615.34
total_reward: 88615.34
total_cost: 68.66
total_trades: 28
Sharpe: 1.285
hit end!


In [23]:
df_actions.to_csv("action.csv", index=False) 
print(f"df_actions: {df_actions}")

df_actions:             600000  600009  600016  600028  600030  600031  600036  600050  \
date                                                                         
2019-08-01    1000    1000       0    1000    1000    1000    1000    1000   
2019-08-01    1000    1000       0    1000    1000    1000    1000    1000   
2019-08-02       0       0       0       0       0       0       0     300   
2019-08-05       0       0       0       0       0       0       0       0   
2019-08-06       0       0       0       0       0       0       0       0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
2019-12-24       0       0       0       0       0       0       0       0   
2019-12-25       0       0       0       0       0       0       0       0   
2019-12-26       0       0       0       0       0       0       0       0   
2019-12-27       0       0       0       0       0       0       0       0   
2019-12-30       0       0       0       0       0  

## Backtest

In [24]:
# 补充time列
df_account_value.time = df_account_value.date

In [25]:
# 对齐两个DataFrame数据
trade["date"] = pd.to_datetime(trade["date"])

account_dates = df_account_value['date'].unique()
trade_filtered = trade[trade['date'].isin(account_dates)]

In [26]:
import plotly.express as px

px.line(df_account_value, x="date", y="account_value", title="Account Value")